### 크롬 드라이버 열기

#### 확장 프로그램(.crx) 적용하기

In [ ]:
# import base64

# # YouTube to Text 확장 프로그램
# crx_file_path = 'downloaded/apnedodbofogffiagpekmbeflilkcbgf.crx'
# with open(crx_file_path, 'rb') as crx_file:
#     encoded_extension = base64.b64encode(crx_file.read()).decode('utf-8')

#### 크롬 드라이버 옵션 설정

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--mute-audio")  # 음소거 옵션 추가
# options.add_encoded_extension(encoded_extension)  # YouTube to Text 확장 프로그램

driver = webdriver.Chrome(options=options)

---

### 로그인 시도

In [24]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import global_constants

# 로그인이 완료되면 유튜브로 넘어가는 url
login_url = "https://accounts.google.com/v3/signin/identifier?checkedDomains=youtube&continue=https%3A%2F%2Fwww.youtube.com%2Fsignin%3Faction_handle_signin%3Dtrue%26app%3Ddesktop%26hl%3Dko%26next%3Dhttps%253A%252F%252Fwww.youtube.com%252F&ddm=1&ec=65620&flowEntry=ServiceLogin&flowName=GlifWebSignIn&hl=ko&ifkv=AcMMx-fxZ82P_dh4l0x5OjBskffPj3UKT340XP69FSCdZDGoG_c21nyWoAp-KMGFQaoU1LvrcTn1_Q&pstMsg=1&service=youtube&dsh=S136025753%3A1730345560396742"
driver.get(login_url)

# 아이디 입력
username_input = driver.find_element(By.CSS_SELECTOR, "input#identifierId.whsOnd.zHQkBf")
username_input.send_keys(global_constants.my_youtube_id)
username_input.send_keys(Keys.RETURN)

In [25]:
# 비밀번호 입력
password_input = driver.find_element(By.CSS_SELECTOR, "input.whsOnd.zHQkBf")
password_input.send_keys(global_constants.my_youtube_pw)
password_input.send_keys(Keys.RETURN)

로그인 완료!

---

### 스크립트 추출에 실패한 비디오 리스트 파일 불러오기

In [ ]:
import pandas as pd

df = pd.read_csv("shared/failed_video_ids_merged_youtube_data.csv")
failed_video_ids = df['videoId'].tolist()  # 중복 없음, 399개

In [ ]:
print(failed_video_ids[:20])

### 순서대로 비디오 열고 스크립트 텍스트 수집하기

In [ ]:
import time

video_scripts = []

for i, failed_video_id in enumerate(failed_video_ids[170:]):
    print("\n")

    failed_video_url = "https://www.youtube.com/watch?v=" + failed_video_id
    driver.get(failed_video_url)
    time.sleep(7)
    
    # discription text: "...더보기" 버튼 클릭
    try:
        more_discription_button = driver.find_element(By.CSS_SELECTOR, 'tp-yt-paper-button#expand.button.style-scope.ytd-text-inline-expander')
        more_discription_button.click()
        print(f"failed_video_id: {failed_video_id}, \"더보기\" 버튼 클릭 완료!!")
        time.sleep(3)
    except Exception as e:
        print(f"failed_video_id: {failed_video_id}, 더보기 버튼 클릭 실패.")

    # "스크립트 표시" 버튼 클릭
    try:
        more_discription_button = driver.find_element(By.CSS_SELECTOR, 'button.yt-spec-button-shape-next.yt-spec-button-shape-next--outline.yt-spec-button-shape-next--call-to-action.yt-spec-button-shape-next--size-m.yt-spec-button-shape-next--enable-backdrop-filter-experiment')
        more_discription_button.click()
        print(f"failed_video_id: {failed_video_id}, \"스크립트 표시\" 버튼 클릭 완료!!")
        time.sleep(3)
        
        # 스크립트 텍스트 수집
        script_texts = []
        try:
            script_text_tags = driver.find_elements(By.CSS_SELECTOR, 'div.segment.style-scope.ytd-transcript-segment-renderer yt-formatted-string.segment-text.style-scope.ytd-transcript-segment-renderer')
            for script_text_tag in script_text_tags:
                script_texts.append(script_text_tag.text)
            print(f"failed_video_id: {failed_video_id}, 스크립트 텍스트 수집 완료!!")
        except Exception as e:
            print("script_texts Error.")
            script_texts = None

    except Exception as e:
        print(f"failed_video_id: {failed_video_id}, 스크립트 정보 없음.")
        script_texts = None

    video_scripts.append([failed_video_id, script_texts])

In [ ]:
# 수집한 url을 csv 파일로 저장
df = pd.DataFrame(video_scripts, columns=["video_id", "script_text"])
df.to_csv("failed_video_script_try(3).csv", index=False)
print("failed_video_script_try(3).csv file saved!")

#### 최종 파일로 합치기

In [ ]:
df1 = pd.read_csv('failed_video_script_try(1).csv')
df2 = pd.read_csv('failed_video_script_try(2).csv')
df3 = pd.read_csv('failed_video_script_try(3).csv')

combined_df = pd.concat([df1, df2, df3])

combined_df.to_csv('failed_video_script.csv', index=False)

#### JSON 형식으로 변환하기

In [ ]:
import pandas as pd

df4 = pd.read_csv('data/failed_video_script_dropna.csv')

output_path = 'data/failed_video_script_dropna.json'
df4.to_json(output_path, orient='records', force_ascii=False, indent=4)
